In [58]:
import sys
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from nltk.tokenize import TreebankWordTokenizer
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix, classification_report

pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = 999
np.set_printoptions(threshold=sys.maxsize)

In [59]:
glove_input_file = '../models/w2v_glove_300.txt'
word2vec_output_file = 'w2v.txt'
glove2word2vec(glove_input_file, word2vec_output_file)
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [60]:
w1 ="stroke"
print(model.most_similar(positive = w1, topn = 6))
#model.most_similar_cosmul(positive=['hepatoma', 'brain'], negative=['liver'])

[('heart', 0.4828336238861084), ('cardiac', 0.45815250277519226), ('debilitating', 0.4523412585258484), ('infarction', 0.4387247562408447), ('illness', 0.4336370527744293), ('complications', 0.43345579504966736)]


In [61]:
file = "../data/stopwords.txt"
with open(file) as f:
    stop_words = f.read().splitlines()

print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [62]:
def sentence_vector(sentence):
    word_list = TreebankWordTokenizer().tokenize(sentence)
    word_list = [word for word in word_list if word not in stop_words]
    word_vectors = []
    for x in word_list:
        try:
            word_vectors.append(model[x])
        except KeyError:
            None    
    return sum(word_vectors)/len(word_vectors)

In [64]:
# load prepartitioned train/test sets
test = pd.read_csv("../data/test.csv")
train = pd.read_csv("../data/AMIA_train_set.csv")

test['vec'] = [sentence_vector(x) for x in test.text]
train['vec'] = [sentence_vector(x) for x in train.text]

# load full data set
frames = [test, train]
df = pd.concat(frames)
df.expansion.unique()

train_grouped_abbr = train.groupby('abbrev')
test_grouped_abbr = test.groupby('abbrev')

In [65]:
X = list(df.vec)
X = np.array(X)
y = df.expansion

X1 = list(train.vec)
X_train = np.array(X1)
y_train = train.expansion

X2 = list(test.vec)
X_test = np.array(X2)
y_test = test.expansion

In [35]:
# Define classifiers:
svc = SVC(C=1.0, kernel='linear', degree=1)
logistic = LogisticRegression()
mlp = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)

In [67]:
# Loop through different abbreviations.
clf_type = 'mlp'
for abbr in train.abbrev.unique():

    train_abbr = train_grouped_abbr.get_group(abbr)
    test_abbr = test_grouped_abbr.get_group(abbr)

    X_train = np.array(list(train_abbr.vec))
    y_train = train_abbr.expansion

    X_test = np.array(list(test_abbr.vec))
    y_test = test_abbr.expansion

    if clf_type == 'svm':
        # set up SVM
        clf = SVC(C=1.0, kernel='linear', degree=1, probability=True).fit(X_train, y_train)

    elif clf_type == 'logistic':
        clf = LogisticRegression().fit(X_train, y_train)
        
    elif clf_type == 'mlp':
        clf = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500).fit(X_train, y_train)
        
    pred = clf.predict(X_test)
    cm = confusion_matrix(y_test, pred, labels=list(set(df.expansion)))
    print()
    print("##" * 20)
    print(" " * 20 + abbr)
    print("##" * 20)

    print(classification_report(y_test, pred))
    print()
    print(f'examples (first 5 cases)\t\t\t\t\t\ttrue_abbr\t\t\tpred_abbr')

    # Print first 5 cases
    i = 0
    for input_row, true_abbr, pred_abbr in zip(train_abbr.iterrows(), y_test, pred):

        sn_start = max(input_row[1].start - 25, 0)
        sn_end = min(input_row[1].end + 25, len(input_row[1].text))

        example_text = input_row[1].text[sn_start: sn_end]
        print(f'... {example_text} ...\t{true_abbr:<35}\t{pred_abbr}')

        if i == 5:
            break

        i += 1

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



########################################
                    MS
########################################
                    precision    recall  f1-score   support

     UNSURED SENSE       0.00      0.00      0.00         1
  morphine sulfate       0.87      0.87      0.87        30
multiple sclerosis       0.75      0.83      0.79        18
   musculoskeletal       0.00      0.00      0.00         1

         micro avg       0.82      0.82      0.82        50
         macro avg       0.40      0.43      0.41        50
      weighted avg       0.79      0.82      0.80        50


examples (first 5 cases)						true_abbr			pred_abbr
... ADMISSION MEDICATIONS: MS Contin as above, predniso ...	morphine sulfate                   	morphine sulfate
... ing. The patient is on a MS Contin and Percocet. On ...	multiple sclerosis                 	multiple sclerosis
... nd is remarkable for: 1. MS. 2. Hypertension. 3. Ast ...	morphine sulfate                   	morphine sulfate
... nd tremor se

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



########################################
                    MR
########################################
                          precision    recall  f1-score   support

         GENERAL ENGLISH       0.00      0.00      0.00         1
      magnetic resonance       0.90      0.93      0.91        28
    mitral regurgitation       0.81      0.85      0.83        20
myocardial infarction:MI       0.00      0.00      0.00         1

               micro avg       0.86      0.86      0.86        50
               macro avg       0.43      0.44      0.44        50
            weighted avg       0.83      0.86      0.84        50


examples (first 5 cases)						true_abbr			pred_abbr
... ail and severe eccentric MR with pulmonary vein rev ...	mitral regurgitation               	mitral regurgitation
... ill proceed Tuesday with MR cholangiogram to assess ...	mitral regurgitation               	mitral regurgitation
... n both cerebellum areas. MR angiography revealed a  ...	mitral regurgita

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



########################################
                    IT
########################################
                        precision    recall  f1-score   support

       GENERAL ENGLISH       0.70      0.89      0.78        18
         UNSURED SENSE       0.00      0.00      0.00         0
            iliotibial       1.00      1.00      1.00         3
information technology       0.85      0.73      0.79        15
     intertrochanteric       1.00      0.50      0.67         2
           intrathecal       0.86      0.67      0.75         9
    ischial tuberosity       1.00      0.67      0.80         3

             micro avg       0.78      0.78      0.78        50
             macro avg       0.77      0.64      0.68        50
          weighted avg       0.82      0.78      0.79        50


examples (first 5 cases)						true_abbr			pred_abbr
... : ASPIRIN AND IBUPROFEN; IT DOES NOT HAVE ANY ALLER ...	intrathecal                        	intrathecal
... employed as a manager 

In [57]:
# bagging test
seed = 1075
np.random.seed(seed)
svc = SVC(C=1.0, kernel='linear', degree=1)
for i in range(1, 11):

    bagging_clf = BaggingClassifier(svc, max_samples=i*(0.1), max_features=10, random_state=seed)
    bagging_scores = cross_val_score(bagging_clf, X, y, cv=7, n_jobs=-1)
    #print(bagging_scores)
    print("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".format(svc.__class__.__name__, 
                        bagging_scores.mean(), bagging_scores.std()))
    
vanilla_scores = cross_val_score(svc, X, y, cv=7, n_jobs=-1)

print("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Vanilla {0}]\n".format(svc.__class__.__name__, 
                        vanilla_scores.mean(), vanilla_scores.std()))


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Mean of: 0.043, std: (+/-) 0.018 [Bagging SVC]

Mean of: 0.040, std: (+/-) 0.014 [Bagging SVC]



/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Mean of: 0.033, std: (+/-) 0.021 [Bagging SVC]



/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


...

Mean of: 0.032, std: (+/-) 0.023 [Bagging SVC]



/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Mean of: 0.058, std: (+/-) 0.055 [Bagging SVC]



/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Mean of: 0.691, std: (+/-) 0.010 [Vanilla SVC]



In [33]:
print('accuracy: {}'.format(cross_val_scores))
print()
#print(set(df.expansion))
#print([len(df[df.expansion == x]) for x in set(df.expansion)])
print()
#print(cm)
print()
print(f1_score(y_test,pred,average = 'weighted'))

accuracy: [0.69942197 0.68712871 0.687251   0.68336673 0.70445344 0.70325203
 0.71574642]



0.6411983035331859


In [ ]:
sentence = 'Patient was tested for US, cystic fibrosis and other heritable diseases '

sentence = "Procedure went without complications, and the patient was sent to the \
floor postoperatively after he was extubated in the FISH. Thoracic surgery was consulted \
on the day of surgery. Gastrografin upper GI study performed on admission showed no leak. \
However, the patient was admitted under the care of thoracic surgery team, and he was kept \
n.p.o. and followed up on daily basis for any change in vital signs, chest pain for another \
upper GI swallow study which was done 7 days after his symptoms started. "


sentence = ['she had an US to determine if the baby was good', 'he had an US to determine if there was a mass', 'If the patient continues to require blood transfusions an/or if he becomes hemodynamically unstable he will need to be taken to the OR for cystoscopy with clot evacuation. Since his surgery was so recently performed we would rather try to hold off on this, however, as not to disrupt the anastomosis with his new transplanted kidney.']

for s in sentence:
    print('sentence:', s)
    #print(vector_breakage(sentence))
    print('prediction:', clf.predict(sentence_vector(s).reshape(1, -1)))
    print()

## Try

In [ ]:
try_out = ''
print(vector_breakage(try_out))
print(clf.predict(sentence_vector(try_out).reshape(1, -1)))